<a href="https://colab.research.google.com/github/sruthi1996/DeepLearning/blob/master/AmazonPlanetModelWithKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai.vision import *

###About the dataset

Dataset used to understand the planet from space. The dataset was taken from kaggle. Contains Train and Test datasets; each as an image name feature as well as set of tags that describe that image; seperated by whitespace.

In [0]:
import pandas as pd
import numpy as np
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
import zipfile
import cv2
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
import keras as k
#from keras.models import Sequentially
from tensorflow.keras import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import datetime

In [0]:
from tensorboard.plugins.hparams import api_pb2
from keras.callbacks import TensorBoard
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext tensorboard
from tensorboard.plugins.hparams import summary as hparams_summary
from google.protobuf import struct_pb2

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id':"1r874VVhJ7eeerIFgZ9ulX79BoRR6QKeW"})
download.GetContentFile('sample_submission_v2.csv')

In [0]:
import pandas as pd
data_test = pd.read_csv('sample_submission_v2.csv')

In [0]:
data_test

,image_name,tags
0,test_0,primary clear agriculture road water
1,test_1,primary clear agriculture road water
2,test_2,primary clear agriculture road water
3,test_3,primary clear agriculture road water
4,test_4,primary clear agriculture road water
...,...,...
61186,file_9995,primary clear agriculture road water
61187,file_9996,primary clear agriculture road water
61188,file_9997,primary clear agriculture road water
61189,file_9998,primary clear agriculture road water


In [0]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61191 entries, 0 to 61190
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_name  61191 non-null  object
 1   tags        61191 non-null  object
dtypes: object(2)
memory usage: 956.2+ KB


In [0]:
download = drive.CreateFile({'id':"172L8Cep0u2EUUwtorzm8ZfWhrDDEi3Z6"})
download.GetContentFile('train_v2.csv')

In [0]:
import pandas as pd
data_train = pd.read_csv('train_v2.csv')

In [0]:
data_train.head(3)

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary


Seperating the tags for each image; using ' ' separater; thus forming labels.

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link = 'https://drive.google.com/open?id=1P6zyoQ7ws6rdrkFj9RUtODrLLNykxXJ4'
fluff, id = link.split('=')

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('train-jpg.zip')

with zipfile.ZipFile('train-jpg.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [0]:
x_train = []
y_train = []
labels = data_train['tags'].str.get_dummies(sep=' ').columns
x_test = []

In [0]:
def flatten(layer):
    result = []
    for sublist in layer:     
        for item in sublist: 
            result.append(item)
    return result

In [0]:
labs = list(set(flatten([layer.split(' ') for layer in data_train['tags'].values])))

In [0]:
lab_max = {layer: i for i, layer in enumerate(labs)}
inv_lab_max = {i: layer for layer, i in lab_max.items()}

In [0]:
for f, tags in tqdm(data_train.values, miniters=5000):
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[lab_max[t]] = 1 
    x_train.append(cv2.resize(img, (32, 32)))
    y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float16) / 255.

100%|██████████| 40479/40479 [00:46<00:00, 866.46it/s]


Split train and test into test as 10000 images and train data as rest.

In [0]:
splitter_pt = 10000
x_train, x_valid, y_train, y_valid = x_train[:splitter_pt], x_train[splitter_pt:], y_train[:splitter_pt], y_train[splitter_pt:]
print(x_train.shape)
print(x_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(10000, 32, 32, 3)
(30479, 32, 32, 3)
(10000, 17)
(30479, 17)


In [0]:
import cv2
#import tensorflow_docs as tfdocs
#import tensorflow_docs.modeling
#import tensorflow_docs.plots
from tqdm import tqdm
import numpy as np
import pandas as pd
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile
from keras.preprocessing import image
import urllib.request
from keras.preprocessing.image import *
import keras
from keras.layers import *

In [0]:
def amazonmodel():
  am = keras.models.Sequential()
  am.add(Conv2D(64, (3, 3), input_shape=(32, 32, 3),activation='relu'))
  am.add(MaxPooling2D(pool_size=(2, 2)))
  am.add(Conv2D(32, (3, 3), activation='relu'))
  am.add(MaxPooling2D(pool_size=(2, 2)))
  am.add(Flatten())
  am.add(Dense(units=128, activation='relu'))
  am.add(Dropout(0.5))
  am.add(Dense(units=17, activation='sigmoid'))
  return am

In [0]:
from keras.callbacks import TensorBoard
import tensorflow as tf
import datetime
import os
!rm -rf ./logs/ 

In [0]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=logdir, histogram_freq=1)

In [0]:
model=amazonmodel()

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
#%tensorboard --logdir fit

In [0]:
model.fit(x_train, y_train, batch_size=128, epochs=15, verbose=1, validation_data=(x_valid, y_valid), callbacks=[tensorboard_callback])

Train on 10000 samples, validate on 30479 samples
Epoch 1/15
10000/10000 [==============================] - 2s 200us/step - loss: 0.3451 - accuracy: 0.8715 - val_loss: 0.2473 - val_accuracy: 0.9054
Epoch 2/15
10000/10000 [==============================] - 2s 171us/step - loss: 0.2587 - accuracy: 0.9028 - val_loss: 0.2313 - val_accuracy: 0.9093
Epoch 3/15
10000/10000 [==============================] - 2s 172us/step - loss: 0.2405 - accuracy: 0.9082 - val_loss: 0.2210 - val_accuracy: 0.9123
Epoch 4/15
10000/10000 [==============================] - 2s 170us/step - loss: 0.2301 - accuracy: 0.9124 - val_loss: 0.2132 - val_accuracy: 0.9172
Epoch 5/15
10000/10000 [==============================] - 2s 171us/step - loss: 0.2241 - accuracy: 0.9146 - val_loss: 0.2094 - val_accuracy: 0.9183
Epoch 6/15
10000/10000 [==============================] - 2s 173us/step - loss: 0.2193 - accuracy: 0.9153 - val_loss: 0.2084 - val_accuracy: 0.9172
Epoch 7/15
10000/10000 [==============================] - 2s 1

In [0]:
pip install -U tensorboard

Requirement already up-to-date: tensorboard in /usr/local/lib/python3.6/dist-packages (2.2.1)


Upload logs onto tensorboard

In [0]:
#!tensorboard dev upload --logdir logs

**Hyperparameter Tuning:**

Techniques like tuning batch size in iterative gradiant descent and number of epochs while training.

- Can Tune learning Rate.

- Can tune Network Weight Initialization

- Activation Function

- Dropout regularization

- Number of neurons in hidden layer

Would Implement few for the am model

Note: Have tried improving accuracy by:

- adding 2 additional dense layers

- changing batch size 32->64->32

-loss functions: mean_squared_error and binary_crossentropy

- used various optimizers like SGD, Adadelta, RMSProp, AdaMax


In [0]:
def amazonmodel2():
  am = keras.models.Sequential()
  am.add(Conv2D(64, (3, 3), input_shape=(32, 32, 3),activation='relu'))
  am.add(MaxPooling2D(pool_size=(2, 2)))
  am.add(Conv2D(32, (3, 3), activation='relu'))
  am.add(MaxPooling2D(pool_size=(2, 2)))
  am.add(Flatten())
  am.add(Dense(units=256, activation='relu'))
  am.add(Dropout(0.5))
  am.add(Dense(units=128, activation='relu'))
  am.add(Dropout(0.5))
  am.add(Dense(units=128, activation='relu'))
  am.add(Dropout(0.5))
  am.add(Dense(units=17, activation='sigmoid'))
  return am

In [0]:
model=amazonmodel2()
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
amazonmodel = model.fit(x_train, y_train,batch_size=128,epochs=15,verbose=1,validation_data=(x_valid, y_valid),callbacks=[tensorboard_callback])

Train on 10000 samples, validate on 30479 samples
Epoch 1/15
10000/10000 [==============================] - 2s 197us/step - loss: 0.6866 - accuracy: 0.5874 - val_loss: 0.6757 - val_accuracy: 0.8458
Epoch 2/15
10000/10000 [==============================] - 2s 168us/step - loss: 0.6675 - accuracy: 0.7170 - val_loss: 0.6539 - val_accuracy: 0.8654
Epoch 3/15
10000/10000 [==============================] - 2s 170us/step - loss: 0.6436 - accuracy: 0.7551 - val_loss: 0.6206 - val_accuracy: 0.8635
Epoch 4/15
10000/10000 [==============================] - 2s 173us/step - loss: 0.6025 - accuracy: 0.7676 - val_loss: 0.5493 - val_accuracy: 0.8452
Epoch 5/15
10000/10000 [==============================] - 2s 168us/step - loss: 0.5268 - accuracy: 0.7921 - val_loss: 0.4230 - val_accuracy: 0.8838
Epoch 6/15
10000/10000 [==============================] - 2s 169us/step - loss: 0.4441 - accuracy: 0.8311 - val_loss: 0.3395 - val_accuracy: 0.9053
Epoch 7/15
10000/10000 [==============================] - 2s 1

It is observed that the accuracy remained constant over all epochs nearly. hence trying with different batchsize, epochs and also the optimizer.

In [0]:
model1=amazonmodel2()
model1.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
amazonmodel = model1.fit(x_train, y_train,batch_size=32,epochs=10,verbose=1,validation_data=(x_valid, y_valid),callbacks=[tensorboard_callback])

Train on 10000 samples, validate on 30479 samples
Epoch 1/10
10000/10000 [==============================] - 4s 449us/step - loss: 0.2766 - accuracy: 0.8967 - val_loss: 0.2260 - val_accuracy: 0.9108
Epoch 2/10
10000/10000 [==============================] - 4s 419us/step - loss: 0.2257 - accuracy: 0.9114 - val_loss: 0.2358 - val_accuracy: 0.9118
Epoch 3/10
10000/10000 [==============================] - 4s 418us/step - loss: 0.2097 - accuracy: 0.9157 - val_loss: 0.1911 - val_accuracy: 0.9241
Epoch 4/10
10000/10000 [==============================] - 4s 419us/step - loss: 0.2013 - accuracy: 0.9192 - val_loss: 0.1863 - val_accuracy: 0.9251
Epoch 5/10
10000/10000 [==============================] - 4s 416us/step - loss: 0.1953 - accuracy: 0.9229 - val_loss: 0.1865 - val_accuracy: 0.9252
Epoch 6/10
10000/10000 [==============================] - 4s 415us/step - loss: 0.1911 - accuracy: 0.9245 - val_loss: 0.1784 - val_accuracy: 0.9291
Epoch 7/10
10000/10000 [==============================] - 4s 4

In [0]:
model2=amazonmodel2()
model2.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
amazonmodel2 = model2.fit(x_train, y_train,batch_size=64,epochs=10,verbose=1,validation_data=(x_valid, y_valid),callbacks=[tensorboard_callback])

Train on 10000 samples, validate on 30479 samples
Epoch 1/10
10000/10000 [==============================] - 3s 294us/step - loss: 0.3015 - accuracy: 0.8898 - val_loss: 0.2385 - val_accuracy: 0.9053
Epoch 2/10
10000/10000 [==============================] - 3s 259us/step - loss: 0.2455 - accuracy: 0.9061 - val_loss: 0.2232 - val_accuracy: 0.9096
Epoch 3/10
10000/10000 [==============================] - 3s 257us/step - loss: 0.2283 - accuracy: 0.9095 - val_loss: 0.2063 - val_accuracy: 0.9165
Epoch 4/10
10000/10000 [==============================] - 3s 263us/step - loss: 0.2155 - accuracy: 0.9119 - val_loss: 0.1968 - val_accuracy: 0.9183
Epoch 5/10
10000/10000 [==============================] - 3s 261us/step - loss: 0.2085 - accuracy: 0.9143 - val_loss: 0.1890 - val_accuracy: 0.9210
Epoch 6/10
10000/10000 [==============================] - 3s 261us/step - loss: 0.2010 - accuracy: 0.9182 - val_loss: 0.2207 - val_accuracy: 0.9106
Epoch 7/10
10000/10000 [==============================] - 3s 2

In [0]:
!tensorboard dev upload --logdir logs

2020-05-05 06:17:26.646622: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Data for the "graphs" plugin is now uploaded to TensorBoard.dev! Note that uploaded data is public. If you do not want to upload data for this plugin, use the "--plugins" command line argument.
Upload started and will continue reading any new data as it's added
to the logdir. To stop uploading, press Ctrl-C.
View your TensorBoard live at: https://tensorboard.dev/experiment/KbkQ8gTYRMKOmpcwexSqEA/
E0505 06:17:30.676089 140252155189120 uploader.py:770] Attempted to re-upload existing blob.  Skipping.
E0505 06:17:31.627651 140252155189120 uploader.py:770] Attempted to re-upload existing blob.  Skipping.
E0505 06:17:32.659365 140252155189120 uploader.py:770] Attempted to re-upload existing blob.  Skipping.

Upload stopped. View your TensorBoard at https://tensorboard.dev/experiment/KbkQ8gTYRMKOmpcwexSqEA/
^C


In [0]:
!tensorboard dev update-metadata --experiment_id 'KbkQ8gTYRMKOmpcwexSqEA' --name "Amazonmodel" --description "Hyperparameter Tuning(Amazon Dataset)"

2020-05-05 06:20:29.543846: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Data for the "graphs" plugin is now uploaded to TensorBoard.dev! Note that uploaded data is public. If you do not want to upload data for this plugin, use the "--plugins" command line argument.
